In [1]:
from pathlib import Path
import sys  
import os

In [2]:
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..", "libs")))
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..", "src")))

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
from libs.Recommenders.SLIM.SLIMElasticNetRecommender import SLIMElasticNetRecommender
from libs.Recommenders.SLIM.SLIM_BPR_Python import SLIM_BPR_Python

import numpy as np
import pandas as pd

In [5]:
from Utils.load_URM import load_URM
from Utils.load_ICM import load_ICM
from libs.Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

URM_all = load_URM("../data/data_train.csv")
ICM_all = load_ICM("../data/data_ICM_metadata.csv")

In [6]:
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)

In [7]:
evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])

EvaluatorHoldout: Ignoring 148 ( 0.4%) Users that have less than 1 test interactions


In [8]:
URM_all_coo = URM_all.tocoo()

slim_model = SLIMElasticNetRecommender(URM_all_coo, verbose=True)
slim_model.fit(l1_ratio=0.112, alpha=0.000255, topK = 300)


SLIMElasticNetRecommender: Processed 7050 (18.5%) in 5.00 min. Items per second: 23.50
SLIMElasticNetRecommender: Processed 14184 (37.2%) in 10.00 min. Items per second: 23.64
SLIMElasticNetRecommender: Processed 21275 (55.8%) in 15.00 min. Items per second: 23.64
SLIMElasticNetRecommender: Processed 27923 (73.2%) in 20.00 min. Items per second: 23.27
SLIMElasticNetRecommender: Processed 35078 (92.0%) in 25.00 min. Items per second: 23.38
SLIMElasticNetRecommender: Processed 38121 (100.0%) in 27.26 min. Items per second: 23.31


In [9]:
result_dict, _ = evaluator_validation.evaluateRecommender(slim_model)

EvaluatorHoldout: Processed 35588 (100.0%) in 15.97 sec. Users per second: 2229


In [11]:
result_dict["MAP"]

cutoff
10    0.0
Name: MAP, dtype: object

In [ ]:
'lambda_i_=0.01', 'lambda_j_=0.0036666666666666666'

In [17]:
URM_all_coo = URM_all.tocoo()

slim_bpr_model = SLIM_BPR_Python(URM_all_coo, verbose=False)
slim_bpr_model.fit(learning_rate=0.001,
                                   lambda_i=0.01,
                                   lambda_j=0.00366,
                                   topK = 100,
                                   epochs=120,)


Train completed in 12.86 minutes


In [23]:
result_dict, _ = evaluator_validation.evaluateRecommender(slim_bpr_model)

EvaluatorHoldout: Processed 35576 (100.0%) in 19.34 sec. Users per second: 1840


In [24]:
result_dict["MAP"]

cutoff
10    0.0
Name: MAP, dtype: object

In [12]:
def write_submission_lib_model(trained_model, filename: str = "submission.csv") -> None:
	"""Builds the submission file from a trained recommender model. The file is saved in a CSV format.

	:param trained_model: A fitted recommender model
	:type trained_model: RecommenderModel
	:param filename: The filename of the submission for this particular recommender model
	:type filename: str
	"""
	target_users_test = pd.read_csv("../data/data_target_users_test.csv",).to_numpy().ravel()

	recommendations = np.array([
		trained_model.recommend(user_id, cutoff=10) for user_id in target_users_test
	])

	if not os.path.exists("../submissions"):
		os.makedirs("../submissions")
	with open(f"../submissions/{filename}", "w") as f:
		f.write("user_id,item_list\n")
		for user_id, recs in zip(target_users_test, recommendations):
			f.write(f"{user_id},{' '.join(map(str, recs))}\n")

In [13]:
write_submission_lib_model(slim_model, filename='slim_urm_300.csv')